# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [11]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) create a view to use with your SQL queries
# 5) write code to answer the quiz questions
spark = SparkSession.builder.appName("SQL quiz").getOrCreate()
user_log = spark.read.json("data/sparkify_log_small.json")

In [12]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [13]:
user_log.createOrReplaceTempView("user_log_table")

# Question 1

Which page did user id ""(empty string) NOT visit?

In [14]:
# TODO: write your code to answer question 1
spark.sql("""
            SELECT Distinct(page)
            FROM user_log_table
            WHERE userId = ""
          """).show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



In [15]:
spark.sql("""
            SELECT Distinct(page)
            FROM user_log_table
            WHERE userId != ""
          """).show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|            Home|
|       Downgrade|
|          Logout|
|   Save Settings|
|           About|
|        Settings|
|        NextSong|
|            Help|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

# Question 3

How many female users do we have in the data set?

In [23]:
# TODO: write your code to answer question 3
spark.sql("""
            SELECT COUNT(Distinct(userId)) AS num_females 
            FROM user_log_table
            WHERE gender = "F"
          """).show()

+-----------+
|num_females|
+-----------+
|        462|
+-----------+



# Question 4

How many songs were played from the most played artist?

In [27]:
# TODO: write your code to answer question 4
spark.sql("""
            SELECT artist, count(song) AS num_played
            FROM user_log_table
            Group by artist
            ORDER BY num_played DESC
            LIMIT 1
          """).show()

+--------+----------+
|  artist|num_played|
+--------+----------+
|Coldplay|        83|
+--------+----------+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [42]:
# TODO: write your code to answer question 5
is_home = spark.sql("""
                    SELECT userId, page, ts, CASE
                        WHEN page = 'Home' THEN 1 
                        ELSE 0 
                        END AS is_home
                    FROM user_log_table
                    WHERE page IN ('Home', 'NextSong')
                  """)

# keep the results in a new view
is_home.createOrReplaceTempView('is_home_table')

In [45]:
cumulative_sum = spark.sql("""
                    SELECT *, SUM(is_home) OVER (
                        PARTITION BY userId
                        ORDER BY ts DESC
                        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                    ) AS period
                    FROM is_home_table
                  """)

# keep the results in a view
cumulative_sum.createOrReplaceTempView("period_table")

In [46]:
spark.sql("""
            SELECT AVG(count_results)
            FROM (
                SELECT COUNT(*) AS count_results
                FROM period_table
                GROUP BY userID, period, page
                HAVING page = 'NextSong'
            ) AS counts
          """).show()

+------------------+
|avg(count_results)|
+------------------+
| 6.898347107438017|
+------------------+

